In [1]:
import openai
import json

In [ ]:
openai.api_key = "your_openai_api_key"

## Training Data Processing

In [3]:
with open('data/Input_Response_Flight_Scenarios.json', 'r') as file:
    data1 = json.load(file)
    
with open('data/Input_Response_Turn_2.json', 'r') as file:
    data2 = json.load(file)

In [21]:
def format_training_data(data):
    formatted_data = []
    
    for entry in data:
        input_blocks = []
        
        for point in entry["input"]:
            block = "\n".join([f"{key}: {value}" for key, value in point.items()])
            input_blocks.append(block)
        
        formatted_input = "\n\n".join(input_blocks)
        response = entry["response"]
        
        formatted_data.append({"input": formatted_input, "response": response})
    
    return formatted_data

In [22]:
formate_data1 = format_training_data(data1)
formate_data2 = format_training_data(data2)

In [23]:
train_file = "data/train_data_flight_turn_v2.jsonl"
with open(train_file, 'w') as jsonl_file:
    for dictionary in formate_data1 + formate_data2:
        message = {"messages": [
            {"role": "system", "content": "Evaluate the flight turn data"},
            {"role": "user", "content": dictionary["input"]},
            {"role": "assistant", "content": dictionary["response"]}
            ]}
        jsonl_file.write(json.dumps(message) + '\n')

In [24]:
## Functions from OpenAi for checking the training data and calculate the tokens
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

data_path = "data/train_data_flight_turn_v2.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)
    
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

Num examples: 100
First example:
{'role': 'system', 'content': 'Evaluate the flight turn data'}
{'role': 'user', 'content': 'Point: 0\nASI: 93\nAI: level\nHeading: 180\nTurn and Slip: turn needle right\nBank Angle: 50\nVSI: 50\n\nPoint: 36\nASI: 85\nAI: level\nHeading: 90\nTurn and Slip: turn needle right\nBank Angle: 40\nVSI: 100\n\nPoint: 72\nASI: 93\nAI: level\nHeading: 180\nTurn and Slip: turn needle left\nBank Angle: 40\nVSI: 100\n\nPoint: 108\nASI: 85\nAI: level\nHeading: 90\nTurn and Slip: turn needle left\nBank Angle: 40\nVSI: -100\n\nPoint: 144\nASI: 93\nAI: level\nHeading: 0\nTurn and Slip: turn needle right\nBank Angle: 30\nVSI: -100\n\nPoint: 180\nASI: 87\nAI: level\nHeading: 270\nTurn and Slip: turn needle right\nBank Angle: 40\nVSI: 100\n\nPoint: 216\nASI: 93\nAI: level\nHeading: 90\nTurn and Slip: turn needle left\nBank Angle: 50\nVSI: 50\n\nPoint: 252\nASI: 100\nAI: level\nHeading: 90\nTurn and Slip: turn needle right\nBank Angle: 60\nVSI: -50\n\nPoint: 288\nASI: 100\nA

## Fine-tune OpenAi Model

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="your_openai_api_key")
# Upload the training file to openai
client.files.create(
  file=open("data/train_data_flight_turn_v2.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-4U8dnyLjfViGUqTrrBRo1K', bytes=141775, created_at=1741898889, filename='train_data_flight_turn_v2.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)

In [26]:
# Check all uploaded files
client.files.list()

SyncPage[FileObject](data=[FileObject(id='file-4U8dnyLjfViGUqTrrBRo1K', bytes=141775, created_at=1741898889, filename='train_data_flight_turn_v2.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None), FileObject(id='file-LJBTJx7y7pushfPM5AFiYc', bytes=135375, created_at=1741898759, filename='train_data_flight_turn.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None), FileObject(id='file-HK7wpbX1p1pQLfYB1BPW86', bytes=26492, created_at=1741742335, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None, expires_at=None), FileObject(id='file-8Rdf6T269rdX4F8Kp2Pxvj', bytes=26544, created_at=1741739996, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None, expires_at=None), FileObject(id='file-1Vp5noGdywWp1nD4U3enuY', bytes=176629, created_at=1741736976, filename='train_data_all-p

In [27]:
# Create fine-tuning job
client.fine_tuning.jobs.create(
    training_file="file-4U8dnyLjfViGUqTrrBRo1K",
    model="gpt-4o-2024-08-06" # gpt-4o-mini-2024-07-18 or gpt-4o-2024-08-06
)

FineTuningJob(id='ftjob-1I9psPmZJWq1VxWbbfkMHuOl', created_at=1741898898, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-RO3Int5VH4kDD0ib79PXZngn', result_files=[], seed=1071881758, status='validating_files', trained_tokens=None, training_file='file-4U8dnyLjfViGUqTrrBRo1K', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, metadata=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [29]:
# Check fine-tuning job
client.fine_tuning.jobs.list()
# or 
client.fine_tuning.jobs.retrieve("ftjob-GYK2mL6TZuBcfCntIhjW10T1")

FineTuningJob(id='ftjob-GYK2mL6TZuBcfCntIhjW10T1', created_at=1741898769, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-RO3Int5VH4kDD0ib79PXZngn', result_files=[], seed=707743265, status='queued', trained_tokens=None, training_file='file-LJBTJx7y7pushfPM5AFiYc', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, metadata=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 2.0}}})

In [30]:
client.fine_tuning.jobs.list_events(
  fine_tuning_job_id="ftjob-1I9psPmZJWq1VxWbbfkMHuOl",
  limit=1
)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-tuOm68cxGkLqw9AwAAMwCJTY', created_at=1741899903, level='info', message='Step 116/300: training loss=0.28', object='fine_tuning.job.event', data={'step': 116, 'train_loss': 0.2811059057712555, 'total_steps': 300, 'train_mean_token_accuracy': 0.95652174949646}, type='metrics')], object='list', has_more=True)